In [2]:
#importing libraries
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
#loading the data
df = pd.read_csv('/Users/syedfaquaruddinquadri/Documents/research_project/NHTS/trippub.csv')

In [4]:
df.head(10)

,HOUSEID,PERSONID,TDTRPNUM,STRTTIME,ENDTIME,TRVLCMIN,TRPMILES,TRPTRANS,TRPACCMP,TRPHHACC,VEHID,TRWAITTM,NUMTRANS,TRACCTM,DROP_PRK,TREGRTM,WHODROVE,WHYFROM,LOOP_TRIP,TRPHHVEH,HHMEMDRV,HH_ONTD,NONHHCNT,NUMONTRP,PSGR_FLG,PUBTRANS,TRIPPURP,DWELTIME,TDWKND,VMT_MILE,DRVR_FLG,WHYTRP1S,ONTD_P1,ONTD_P2,ONTD_P3,ONTD_P4,ONTD_P5,ONTD_P6,ONTD_P7,ONTD_P8,ONTD_P9,ONTD_P10,ONTD_P11,ONTD_P12,ONTD_P13,TDCASEID,TRACC_WLK,TRACC_POV,TRACC_BUS,TRACC_CRL,TRACC_SUB,TRACC_OTH,TREGR_WLK,TREGR_POV,TREGR_BUS,TREGR_CRL,TREGR_SUB,TREGR_OTH,WHYTO,TRAVDAY,HOMEOWN,HHSIZE,HHVEHCNT,HHFAMINC,DRVRCNT,HHSTATE,HHSTFIPS,NUMADLT,WRKCOUNT,TDAYDATE,HHRESP,LIF_CYC,MSACAT,MSASIZE,RAIL,URBAN,URBANSIZE,URBRUR,GASPRICE,CENSUS_D,CENSUS_R,CDIVMSAR,HH_RACE,HH_HISP,HH_CBSA,SMPLSRCE,R_AGE,EDUC,R_SEX,PRMACT,PROXY,WORKER,DRIVER,WTTRDFIN,WHYTRP90,TRPMILAD,R_AGE_IMP,R_SEX_IMP,VEHTYPE,OBHUR,DBHUR,OTHTNRNT,OTPPOPDN,OTRESDN,OTEEMPDN,OBHTNRNT,OBPPOPDN,OBRESDN,DTHTNRNT,DTPPOPDN,DTRESDN,DTEEMPDN,DBHTNRNT,DBPPOPDN,DBRESDN
0,30000007,1,1,1000,1015,15,5.244,3,0,0,3,-1,-1,-1,-1,-1,1,1,2,1,1,1,0,1,2,2,HBO,295,2,5.244,1,20,1,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070101,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,19,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,67,3,2,6,1,2,1,75441.905796,5,5.847584,67,2,1,T,T,50,1500,750,750,20,750,300,50,750,300,350,30,300,300
1,30000007,1,2,1510,1530,20,5.149,3,0,0,3,-1,-1,-1,-1,-1,1,19,2,1,1,1,0,1,2,2,HBO,-9,2,5.149,1,1,1,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070102,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,67,3,2,6,1,2,1,75441.905796,5,5.741650,67,2,1,T,T,50,750,300,350,30,300,300,50,1500,750,750,20,750,300
2,30000007,2,1,700,900,120,84.004,6,0,0,5,-1,-1,-1,-1,-1,2,3,2,1,1,1,0,1,2,2,HBW,540,2,84.004,1,1,2,1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070201,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,66,3,1,1,2,1,1,71932.645806,1,90.178294,66,1,4,S,T,40,1500,750,750,40,1500,750,50,1500,750,750,20,750,300
3,30000007,2,2,1800,2030,150,81.628,6,0,0,5,-1,-1,-1,-1,-1,2,1,2,1,1,1,0,1,2,2,HBW,-9,2,81.628,1,10,2,1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070202,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,3,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,66,3,1,1,2,1,1,71932.645806,1,87.627658,66,1,4,T,S,50,1500,750,750,20,750,300,40,1500,750,750,40,1500,750
4,30000007,3,1,845,900,15,2.250,3,0,0,1,-1,-1,-1,-1,-1,3,1,2,1,1,1,0,1,2,2,HBO,330,2,2.250,1,20,2,2,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070301,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,28,2,2,5,2,2,1,80122.686739,5,2.508975,28,2,1,T,T,50,1500,750,750,20,750,300,50,750,300,350,60,750,300
5,30000007,3,2,1430,1445,15,2.243,3,0,0,1,-1,-1,-1,-1,-1,3,8,2,1,1,1,0,1,2,2,HBO,-9,2,2.243,1,1,2,2,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070302,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,28,2,2,5,2,2,1,80122.686739,5,2.501169,28,2,1,T,T,50,750,300,350,60,750,300,50,1500,750,750,20,750,300
6,30000008,1,1,1115,1130,15,8.017,6,0,0,4,-1,-1,-1,-1,-1,1,1,2,1,1,1,0,1,2,2,HBW,720,2,8.017,1,10,1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000080101,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,3,5,1,2,4,8,2,WI,55,2,2,201608,2,2,2,5,2,4,6,2,225.9,3,2,32,1,2,33460,2,55,5,1,1,1,1,1,23062.857428,1,8.606250,55,1,4,R,T,5,300,300,150,5,300,300,30,300,300,150,40,1500,750
7,30000008,1,2,2330,2340,10,8.017,6,0,0,4,-1,-1,-1,-1,-1,1,3,2,1,1,1,0,1,2,2,HBW,-9,2,8.017,1,1,1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000080102,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,5,1,2,4,8,2,WI,55,2,2,201608,2,2,2,5,2,4,6,2,225.9,3,2,32,1,2,33460,2,55,5,1,1,1,1,1,23062.857428,1,8.606250,55,1,4,T,R,30,300,300,150,40,1500,750,5,300,300,150,5,300,300
8,30000012,1,1,550,605,15,3.395,4,0,0,2,-1,-1,-1,-1,-1,1,1,2,1,1,1,0,1,2,2,HBSOCREC,55,2,3.395,1,50,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000120101,-1,-1

In [5]:
#function to filter out the state and urban status
def filter_state(df, state_code):
    state = df[df['HHSTFIPS'] == state_code]
    urban_state = state[state['URBAN'].isin([1,2,3])]
    return urban_state

In [6]:
ny_state = filter_state(df, 36)
nj_state = filter_state(df, 34)

In [22]:
ny_state.shape

(83369, 115)

In [7]:
ny_state['CENSUS_D'].unique()
ny_state['CENSUS_D'].value_counts()

CENSUS_D
2    83330
5       24
1        8
3        7
Name: count, dtype: int64

In [8]:
nj_state['CENSUS_R'].value_counts()

CENSUS_R
1    3907
3       6
4       4
2       1
Name: count, dtype: int64

### activity chain 
##### activity and their code as per 'WHYTRP1S'
- H: in home             - 01
- W: work                - 10
- M: maintenance         - 30
- S: School              - 20
- K: pickup/drop off     - 70
- O: other               - 97
- R: recreational/social - 50
- P: personal activities - 40

In [9]:
#creating a new column for the activity chain of the trip as per WHYTRP1S
activity_mapping = {
    1 : 'H',
    10 : 'W',
    20 : 'S',
    30 : 'M',
    40 : 'P',
    50 : 'R',
    70 : 'K',
    97 : 'O'
}


df['activity'] = df['WHYTRP1S'].map(activity_mapping)

print(df['activity'])

0         S
1         H
2         H
3         W
4         S
         ..
923567    W
923568    R
923569    W
923570    P
923571    H
Name: activity, Length: 923572, dtype: object


In [10]:
df.head(10)

,HOUSEID,PERSONID,TDTRPNUM,STRTTIME,ENDTIME,TRVLCMIN,TRPMILES,TRPTRANS,TRPACCMP,TRPHHACC,VEHID,TRWAITTM,NUMTRANS,TRACCTM,DROP_PRK,TREGRTM,WHODROVE,WHYFROM,LOOP_TRIP,TRPHHVEH,HHMEMDRV,HH_ONTD,NONHHCNT,NUMONTRP,PSGR_FLG,PUBTRANS,TRIPPURP,DWELTIME,TDWKND,VMT_MILE,DRVR_FLG,WHYTRP1S,ONTD_P1,ONTD_P2,ONTD_P3,ONTD_P4,ONTD_P5,ONTD_P6,ONTD_P7,ONTD_P8,ONTD_P9,ONTD_P10,ONTD_P11,ONTD_P12,ONTD_P13,TDCASEID,TRACC_WLK,TRACC_POV,TRACC_BUS,TRACC_CRL,TRACC_SUB,TRACC_OTH,TREGR_WLK,TREGR_POV,TREGR_BUS,TREGR_CRL,TREGR_SUB,TREGR_OTH,WHYTO,TRAVDAY,HOMEOWN,HHSIZE,HHVEHCNT,HHFAMINC,DRVRCNT,HHSTATE,HHSTFIPS,NUMADLT,WRKCOUNT,TDAYDATE,HHRESP,LIF_CYC,MSACAT,MSASIZE,RAIL,URBAN,URBANSIZE,URBRUR,GASPRICE,CENSUS_D,CENSUS_R,CDIVMSAR,HH_RACE,HH_HISP,HH_CBSA,SMPLSRCE,R_AGE,EDUC,R_SEX,PRMACT,PROXY,WORKER,DRIVER,WTTRDFIN,WHYTRP90,TRPMILAD,R_AGE_IMP,R_SEX_IMP,VEHTYPE,OBHUR,DBHUR,OTHTNRNT,OTPPOPDN,OTRESDN,OTEEMPDN,OBHTNRNT,OBPPOPDN,OBRESDN,DTHTNRNT,DTPPOPDN,DTRESDN,DTEEMPDN,DBHTNRNT,DBPPOPDN,DBRESDN,activity
0,30000007,1,1,1000,1015,15,5.244,3,0,0,3,-1,-1,-1,-1,-1,1,1,2,1,1,1,0,1,2,2,HBO,295,2,5.244,1,20,1,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070101,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,19,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,67,3,2,6,1,2,1,75441.905796,5,5.847584,67,2,1,T,T,50,1500,750,750,20,750,300,50,750,300,350,30,300,300,S
1,30000007,1,2,1510,1530,20,5.149,3,0,0,3,-1,-1,-1,-1,-1,1,19,2,1,1,1,0,1,2,2,HBO,-9,2,5.149,1,1,1,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070102,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,67,3,2,6,1,2,1,75441.905796,5,5.741650,67,2,1,T,T,50,750,300,350,30,300,300,50,1500,750,750,20,750,300,H
2,30000007,2,1,700,900,120,84.004,6,0,0,5,-1,-1,-1,-1,-1,2,3,2,1,1,1,0,1,2,2,HBW,540,2,84.004,1,1,2,1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070201,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,66,3,1,1,2,1,1,71932.645806,1,90.178294,66,1,4,S,T,40,1500,750,750,40,1500,750,50,1500,750,750,20,750,300,H
3,30000007,2,2,1800,2030,150,81.628,6,0,0,5,-1,-1,-1,-1,-1,2,1,2,1,1,1,0,1,2,2,HBW,-9,2,81.628,1,10,2,1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070202,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,3,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,66,3,1,1,2,1,1,71932.645806,1,87.627658,66,1,4,T,S,50,1500,750,750,20,750,300,40,1500,750,750,40,1500,750,W
4,30000007,3,1,845,900,15,2.250,3,0,0,1,-1,-1,-1,-1,-1,3,1,2,1,1,1,0,1,2,2,HBO,330,2,2.250,1,20,2,2,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070301,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,28,2,2,5,2,2,1,80122.686739,5,2.508975,28,2,1,T,T,50,1500,750,750,20,750,300,50,750,300,350,60,750,300,S
5,30000007,3,2,1430,1445,15,2.243,3,0,0,1,-1,-1,-1,-1,-1,3,8,2,1,1,1,0,1,2,2,HBO,-9,2,2.243,1,1,2,2,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070302,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,28,2,2,5,2,2,1,80122.686739,5,2.501169,28,2,1,T,T,50,750,300,350,60,750,300,50,1500,750,750,20,750,300,H
6,30000008,1,1,1115,1130,15,8.017,6,0,0,4,-1,-1,-1,-1,-1,1,1,2,1,1,1,0,1,2,2,HBW,720,2,8.017,1,10,1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000080101,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,3,5,1,2,4,8,2,WI,55,2,2,201608,2,2,2,5,2,4,6,2,225.9,3,2,32,1,2,33460,2,55,5,1,1,1,1,1,23062.857428,1,8.606250,55,1,4,R,T,5,300,300,150,5,300,300,30,300,300,150,40,1500,750,W
7,30000008,1,2,2330,2340,10,8.017,6,0,0,4,-1,-1,-1,-1,-1,1,3,2,1,1,1,0,1,2,2,HBW,-9,2,8.017,1,1,1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000080102,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,5,1,2,4,8,2,WI,55,2,2,201608,2,2,2,5,2,4,6,2,225.9,3,2,32,1,2,33460,2,55,5,1,1,1,1,1,23062.857428,1,8.606250,55,1,4,T,R,30,300,300,150,40,1500,750,5,300,300,150,5,300,300,H
8,30000012,1,1,550,605,15,3.395,4,0,0,2,-1,-1,-1,-1,-1,1,1,2,1,1,1,0,1,2,2,HBSOCREC,55,2,3.395,1,50,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1

In [43]:
#activity chain for each person in the each household in the dataset
def generate_activity_chain(df):
    activity_chain_dict = {}
    
    # Iterate over each unique combination of HOUSEID and PERSONID
    for (house_id, person_id), group in df.groupby(['HOUSEID', 'PERSONID']):
        activity_chain = ''
        # Iterate over each row in the group
        for index, row in group.iterrows():
            # Map the activity to its corresponding letter
            activity = row['activity']
            activity_letter = {
                'home': 'H',
                'work': 'W',
                'school': 'S',
                'maintenance': 'M',
                'pick up/drop off': 'K',
                'other': 'O',
                'recreational/social': 'R',
                'personal activities': 'P'
            }.get(activity, 'O')  # Default to 'O' if activity is not found
            
            # Append the activity letter to the activity chain
            activity_chain += activity_letter
        
        # Add the activity chain to the dictionary with the person_id as the key
        activity_chain_dict[(house_id, person_id)] = activity_chain
    
    return activity_chain_dict

# Call the function with your DataFrame
activity_chain_dict = generate_activity_chain(ny_state)

# Print the result
print(activity_chain_dict)


KeyError: 'activity'

In [11]:
#filtering all the rows for particular household
HH1 = df[df['HOUSEID'] == 30000007]
HH1.head(10)

,HOUSEID,PERSONID,TDTRPNUM,STRTTIME,ENDTIME,TRVLCMIN,TRPMILES,TRPTRANS,TRPACCMP,TRPHHACC,VEHID,TRWAITTM,NUMTRANS,TRACCTM,DROP_PRK,TREGRTM,WHODROVE,WHYFROM,LOOP_TRIP,TRPHHVEH,HHMEMDRV,HH_ONTD,NONHHCNT,NUMONTRP,PSGR_FLG,PUBTRANS,TRIPPURP,DWELTIME,TDWKND,VMT_MILE,DRVR_FLG,WHYTRP1S,ONTD_P1,ONTD_P2,ONTD_P3,ONTD_P4,ONTD_P5,ONTD_P6,ONTD_P7,ONTD_P8,ONTD_P9,ONTD_P10,ONTD_P11,ONTD_P12,ONTD_P13,TDCASEID,TRACC_WLK,TRACC_POV,TRACC_BUS,TRACC_CRL,TRACC_SUB,TRACC_OTH,TREGR_WLK,TREGR_POV,TREGR_BUS,TREGR_CRL,TREGR_SUB,TREGR_OTH,WHYTO,TRAVDAY,HOMEOWN,HHSIZE,HHVEHCNT,HHFAMINC,DRVRCNT,HHSTATE,HHSTFIPS,NUMADLT,WRKCOUNT,TDAYDATE,HHRESP,LIF_CYC,MSACAT,MSASIZE,RAIL,URBAN,URBANSIZE,URBRUR,GASPRICE,CENSUS_D,CENSUS_R,CDIVMSAR,HH_RACE,HH_HISP,HH_CBSA,SMPLSRCE,R_AGE,EDUC,R_SEX,PRMACT,PROXY,WORKER,DRIVER,WTTRDFIN,WHYTRP90,TRPMILAD,R_AGE_IMP,R_SEX_IMP,VEHTYPE,OBHUR,DBHUR,OTHTNRNT,OTPPOPDN,OTRESDN,OTEEMPDN,OBHTNRNT,OBPPOPDN,OBRESDN,DTHTNRNT,DTPPOPDN,DTRESDN,DTEEMPDN,DBHTNRNT,DBPPOPDN,DBRESDN,activity
0,30000007,1,1,1000,1015,15,5.244,3,0,0,3,-1,-1,-1,-1,-1,1,1,2,1,1,1,0,1,2,2,HBO,295,2,5.244,1,20,1,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070101,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,19,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,67,3,2,6,1,2,1,75441.905796,5,5.847584,67,2,1,T,T,50,1500,750,750,20,750,300,50,750,300,350,30,300,300,S
1,30000007,1,2,1510,1530,20,5.149,3,0,0,3,-1,-1,-1,-1,-1,1,19,2,1,1,1,0,1,2,2,HBO,-9,2,5.149,1,1,1,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070102,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,67,3,2,6,1,2,1,75441.905796,5,5.741650,67,2,1,T,T,50,750,300,350,30,300,300,50,1500,750,750,20,750,300,H
2,30000007,2,1,700,900,120,84.004,6,0,0,5,-1,-1,-1,-1,-1,2,3,2,1,1,1,0,1,2,2,HBW,540,2,84.004,1,1,2,1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070201,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,66,3,1,1,2,1,1,71932.645806,1,90.178294,66,1,4,S,T,40,1500,750,750,40,1500,750,50,1500,750,750,20,750,300,H
3,30000007,2,2,1800,2030,150,81.628,6,0,0,5,-1,-1,-1,-1,-1,2,1,2,1,1,1,0,1,2,2,HBW,-9,2,81.628,1,10,2,1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070202,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,3,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,66,3,1,1,2,1,1,71932.645806,1,87.627658,66,1,4,T,S,50,1500,750,750,20,750,300,40,1500,750,750,40,1500,750,W
4,30000007,3,1,845,900,15,2.250,3,0,0,1,-1,-1,-1,-1,-1,3,1,2,1,1,1,0,1,2,2,HBO,330,2,2.250,1,20,2,2,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070301,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,28,2,2,5,2,2,1,80122.686739,5,2.508975,28,2,1,T,T,50,1500,750,750,20,750,300,50,750,300,350,60,750,300,S
5,30000007,3,2,1430,1445,15,2.243,3,0,0,1,-1,-1,-1,-1,-1,3,8,2,1,1,1,0,1,2,2,HBO,-9,2,2.243,1,1,2,2,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070302,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,28,2,2,5,2,2,1,80122.686739,5,2.501169,28,2,1,T,T,50,750,300,350,60,750,300,50,1500,750,750,20,750,300,H


In [12]:
HH_group = HH1.groupby(['HOUSEID', 'PERSONID'])
HH_group.head(10)
    

,HOUSEID,PERSONID,TDTRPNUM,STRTTIME,ENDTIME,TRVLCMIN,TRPMILES,TRPTRANS,TRPACCMP,TRPHHACC,VEHID,TRWAITTM,NUMTRANS,TRACCTM,DROP_PRK,TREGRTM,WHODROVE,WHYFROM,LOOP_TRIP,TRPHHVEH,HHMEMDRV,HH_ONTD,NONHHCNT,NUMONTRP,PSGR_FLG,PUBTRANS,TRIPPURP,DWELTIME,TDWKND,VMT_MILE,DRVR_FLG,WHYTRP1S,ONTD_P1,ONTD_P2,ONTD_P3,ONTD_P4,ONTD_P5,ONTD_P6,ONTD_P7,ONTD_P8,ONTD_P9,ONTD_P10,ONTD_P11,ONTD_P12,ONTD_P13,TDCASEID,TRACC_WLK,TRACC_POV,TRACC_BUS,TRACC_CRL,TRACC_SUB,TRACC_OTH,TREGR_WLK,TREGR_POV,TREGR_BUS,TREGR_CRL,TREGR_SUB,TREGR_OTH,WHYTO,TRAVDAY,HOMEOWN,HHSIZE,HHVEHCNT,HHFAMINC,DRVRCNT,HHSTATE,HHSTFIPS,NUMADLT,WRKCOUNT,TDAYDATE,HHRESP,LIF_CYC,MSACAT,MSASIZE,RAIL,URBAN,URBANSIZE,URBRUR,GASPRICE,CENSUS_D,CENSUS_R,CDIVMSAR,HH_RACE,HH_HISP,HH_CBSA,SMPLSRCE,R_AGE,EDUC,R_SEX,PRMACT,PROXY,WORKER,DRIVER,WTTRDFIN,WHYTRP90,TRPMILAD,R_AGE_IMP,R_SEX_IMP,VEHTYPE,OBHUR,DBHUR,OTHTNRNT,OTPPOPDN,OTRESDN,OTEEMPDN,OBHTNRNT,OBPPOPDN,OBRESDN,DTHTNRNT,DTPPOPDN,DTRESDN,DTEEMPDN,DBHTNRNT,DBPPOPDN,DBRESDN,activity
0,30000007,1,1,1000,1015,15,5.244,3,0,0,3,-1,-1,-1,-1,-1,1,1,2,1,1,1,0,1,2,2,HBO,295,2,5.244,1,20,1,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070101,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,19,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,67,3,2,6,1,2,1,75441.905796,5,5.847584,67,2,1,T,T,50,1500,750,750,20,750,300,50,750,300,350,30,300,300,S
1,30000007,1,2,1510,1530,20,5.149,3,0,0,3,-1,-1,-1,-1,-1,1,19,2,1,1,1,0,1,2,2,HBO,-9,2,5.149,1,1,1,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070102,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,67,3,2,6,1,2,1,75441.905796,5,5.741650,67,2,1,T,T,50,750,300,350,30,300,300,50,1500,750,750,20,750,300,H
2,30000007,2,1,700,900,120,84.004,6,0,0,5,-1,-1,-1,-1,-1,2,3,2,1,1,1,0,1,2,2,HBW,540,2,84.004,1,1,2,1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070201,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,66,3,1,1,2,1,1,71932.645806,1,90.178294,66,1,4,S,T,40,1500,750,750,40,1500,750,50,1500,750,750,20,750,300,H
3,30000007,2,2,1800,2030,150,81.628,6,0,0,5,-1,-1,-1,-1,-1,2,1,2,1,1,1,0,1,2,2,HBW,-9,2,81.628,1,10,2,1,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070202,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,3,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,66,3,1,1,2,1,1,71932.645806,1,87.627658,66,1,4,T,S,50,1500,750,750,20,750,300,40,1500,750,750,40,1500,750,W
4,30000007,3,1,845,900,15,2.250,3,0,0,1,-1,-1,-1,-1,-1,3,1,2,1,1,1,0,1,2,2,HBO,330,2,2.250,1,20,2,2,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070301,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,28,2,2,5,2,2,1,80122.686739,5,2.508975,28,2,1,T,T,50,1500,750,750,20,750,300,50,750,300,350,60,750,300,S
5,30000007,3,2,1430,1445,15,2.243,3,0,0,1,-1,-1,-1,-1,-1,3,8,2,1,1,1,0,1,2,2,HBO,-9,2,2.243,1,1,2,2,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,300000070302,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,2,1,3,5,7,3,NC,37,3,1,201608,1,10,3,1,2,1,1,1,228.4,5,3,53,2,2,XXXXX,2,28,2,2,5,2,2,1,80122.686739,5,2.501169,28,2,1,T,T,50,750,300,350,60,750,300,50,1500,750,750,20,750,300,H


In [15]:
for (household_id, person_id), group_data in HH_group:
    print(f"Household ID: {household_id}, Person ID: {person_id}")
    # Analyze activity patterns for the person
    # For example, calculate frequency of each trip purpose, mode of transportation, etc.
    activity_counts = group_data['WHYTRP1S'].value_counts()
    mode_counts = group_data['TRPTRANS'].value_counts()
    
    # Print or store the activity patterns
    print("Activity counts:")
    print(activity_counts)
    print("Mode counts:")
    print(mode_counts)
    print("-----------------------------")

Household ID: 30000007, Person ID: 1
Activity counts:
WHYTRP1S
20    1
1     1
Name: count, dtype: int64
Mode counts:
TRPTRANS
3    2
Name: count, dtype: int64
-----------------------------
Household ID: 30000007, Person ID: 2
Activity counts:
WHYTRP1S
1     1
10    1
Name: count, dtype: int64
Mode counts:
TRPTRANS
6    2
Name: count, dtype: int64
-----------------------------
Household ID: 30000007, Person ID: 3
Activity counts:
WHYTRP1S
20    1
1     1
Name: count, dtype: int64
Mode counts:
TRPTRANS
3    2
Name: count, dtype: int64
-----------------------------


In [18]:
HH1_group = df.groupby(['HOUSEID', 'PERSONID'])

In [33]:
def generate_activity_chain(person_data):
    activity_chain = " "
    for index, row in person_data.iterrows():
        activity_code = row['WHYTRP1S']
        if activity_code in activity_mapping:
            activity_chain += activity_mapping[activity_code]
    return activity_chain

# Group data by household and person
HH_group = HH1.groupby(['HOUSEID', 'PERSONID'])

# Iterate over each household and person
for (household_id, person_id), group_data in HH_group:
    # Generate activity chain for the person
    activity_chain = generate_activity_chain(group_data)
    print(f"Household ID: {household_id}, Person ID: {person_id}, Activity Chain: {activity_chain}")

Household ID: 30000007, Person ID: 1, Activity Chain:  SH
Household ID: 30000007, Person ID: 2, Activity Chain:  HW
Household ID: 30000007, Person ID: 3, Activity Chain:  SH


In [20]:
activity_chain

'WRWPH'

In [22]:

import time
def edit_distance(s1, s2):
    m, n = len(s1), len(s2)
    
    # Initialize a matrix to store edit distances
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    # Initialize the first row and column
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j
    
    # Fill the matrix
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i - 1] == s2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]  # No operation needed
            else:
                dp[i][j] = 1 + min(dp[i - 1][j],      # Deletion
                                   dp[i][j - 1],      # Insertion
                                   dp[i - 1][j - 1])  # Substitution
    
    # Return the edit distance
    return dp[m][n]

# Example usage
s1 = "HHHHHSSSSSWWWWWWWHHHHHHH"
s2 = "HHHHHWWWWWWWWWSSSSSOOOOOHHHHH"
starttime = time.start()

distance = edit_distance(s1, s2)

endtime = time.end()
print("Edit distance between '{}' and '{}' is: {}".format(s1, s2, distance))


Edit distance between 'Recurrence' and 'sitting' is: 9


In [26]:
len(s1)

10

In [31]:
%%time
distance = edit_distance(s1, s2)
print("Edit distance between '{}' and '{}' is: {}".format(s1, s2, distance))

Edit distance between 'Recurrence' and 'sitting' is: 9
CPU times: user 264 µs, sys: 154 µs, total: 418 µs
Wall time: 409 µs


In [32]:
(2.2 * 84000 * 42000)/3600/24

89833.33333333336